In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
v1=np.random.randint(50, 60, (5))
v2=np.random.randint(60, 70, (5))
v3=np.random.randint(50, 60, (5))
value=np.hstack([v1, v2, v3])
l1=np.zeros((5), int)
l2=np.ones((5), int)
l3=np.zeros((5), int)
label=np.hstack([l1, l2, l3])
data=pd.DataFrame({'value':value, 'abnorm':[0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0]})
data

value  abnorm
0      54       0
1      52       0
2      54       1
3      59       1
4      51       1
5      66       0
6      66       1
7      63       0
8      60       0
9      67       1
10     58       0
11     51       1
12     56       1
13     59       0
14     58       0

In [5]:
def make_data(x, str,w):
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)

    return x

def make_label(x, w):
    window=w
    x['label_all']=x['label'].copy()
    for i in range(1, window):
        x.loc[(x['label_all']==1)|(x['label'].shift(i)==1), 'label_all']=1
        x.loc[(x['label_all']==0)&(x['label'].shift(i)==0), 'label_all']=0
    
    x['label']=x['label_all'].copy()
    return x[['label']]
def calc_abnorm(x, w):
    return x.rolling(w, min_periods=1).mean()

In [7]:
window=3
d=calc_abnorm(data[['abnorm']], window)
display(data, d)

value  abnorm
0      54       0
1      52       0
2      54       1
3      59       1
4      51       1
5      66       0
6      66       1
7      63       0
8      60       0
9      67       1
10     58       0
11     51       1
12     56       1
13     59       0
14     58       0

abnorm
0   0.000000
1   0.000000
2   0.333333
3   0.666667
4   1.000000
5   0.666667
6   0.666667
7   0.333333
8   0.333333
9   0.333333
10  0.333333
11  0.666667
12  0.666667
13  0.666667
14  0.333333

In [28]:
df_list=[]
for i in set(data['day']):
    lag=[] 
    window=3
    lag.append(make_data(data.loc[data['day']==i ,['value']], 'value', window))
    lag.append(make_label(data.loc[data['day']==i, ['label']], window))
    lag.append(data.loc[data['day']==i ,['day']])
    df_add_lag=pd.concat(lag, axis=1)
    df_list.append(df_add_lag)

df=pd.concat(df_list)
display(data, df)

value  label  day
0      52      0    0
1      59      0    0
2      52      1    0
3      59      0    0
4      55      1    0
5      61      0    0
6      63      1    0
7      68      0    1
8      66      0    1
9      66      1    1
10     56      0    1
11     55      1    1
12     59      0    1
13     55      0    1
14     50      0    1

value  value_lag1  value_lag2  label  day
0      52         NaN         NaN      0    0
1      59        52.0         NaN      0    0
2      52        59.0        52.0      1    0
3      59        52.0        59.0      1    0
4      55        59.0        52.0      1    0
5      61        55.0        59.0      1    0
6      63        61.0        55.0      1    0
7      68         NaN         NaN      0    1
8      66        68.0         NaN      0    1
9      66        66.0        68.0      1    1
10     56        66.0        66.0      1    1
11     55        56.0        66.0      1    1
12     59        55.0        56.0      1    1
13     55        59.0        55.0      1    1
14     50        55.0        59.0      0    1

In [15]:
df=df.dropna().reset_index(drop=True)
display(df)

value  value_lag1  value_lag2  label
0      53        56.0        54.0      0
1      50        53.0        56.0      0
2      57        50.0        53.0      0
3      61        57.0        50.0      1
4      68        61.0        57.0      1
5      68        65.0        60.0      1
6      58        68.0        65.0      1
7      51        58.0        68.0      1
8      58        51.0        58.0      0
9      55        58.0        51.0      0
10     54        55.0        58.0      0